## Introduction

This tutorial showcases how to use Kvikio to accelerate the loading of DICOM images. We will also utilize the `pydicom` library to handle these medical image formats.

### Common Medical Image Formats

Medical images are complex due to the extensive metadata they contain, which includes patient information, imaging parameters, and more. DICOM (Digital Imaging and Communications in Medicine) is one of the most common formats:

- **Description**: The most widely used standard for storing and transmitting medical images. It includes metadata about the patient, imaging parameters, and more.
- **Usage**: Commonly used in hospitals and clinics for storing images from modalities like MRI, CT, X-ray, and ultrasound.

### Extra Library Used

#### pydicom
- **Description**: A Python library for working with DICOM files. It allows for reading, modifying, and writing DICOM data.
- **Usage**: Widely used in clinical and research settings to handle DICOM files.

### GPU Acceleration with Kvikio

Kvikio is a powerful tool that leverages GPU acceleration to significantly speed up the loading and processing of medical images. In this tutorial, we will demonstrate how to use Kvikio to efficiently handle DICOM images, providing a performance comparison between CPU and GPU processing.

By the end of this tutorial, you will understand:
- How to load DICOM images using `pydicom`.
- How to accelerate the loading and processing of these images using Kvikio.
- The performance benefits of using GPU acceleration for medical image processing.

### Setup Environment

In [1]:
# Check if pydicom is installed, if not, install it
!python -c "import pydicom" || pip install -q pydicom

In [ ]:
import datetime
import os
import shutil
import tempfile
from timeit import default_timer as timer

import cupy as cp
import numpy as np
import pydicom
from pydicom.dataset import Dataset, FileDataset

import kvikio
import kvikio.defaults

### Warmup Kvikio

In [3]:
def warmup_kvikio():
    """
    Warm up the Kvikio library to initialize the internal buffers, cuFile, GDS, etc.
    """
    # warmup cuFile
    a = cp.arange(100)
    with tempfile.NamedTemporaryFile() as tmp_file:
        tmp_file_name = tmp_file.name
        f = kvikio.CuFile(tmp_file_name, "w")
        f.write(a)
        f.close()

        b = cp.empty_like(a)
        f = kvikio.CuFile(tmp_file_name, "r")
        f.read(b)

    # warmup cupy
    c = cp.random.rand(100, 100, 3)
    d = cp.mean(c)


warmup_kvikio()

### Set Kvikio Threads

KvikIO can automatically use multiple threads for I/O operations. Setting the environment variable `KVIKIO_NTHREADS` to the desired number of threads may improve performance. In this tutorial, 4 threads are used. For more details, refer to the [official documentation](https://docs.rapids.ai/api/kvikio/nightly/runtime_settings/#thread-pool-kvikio-nthreads).

In [4]:
kvikio.defaults.num_threads_reset(nthreads=4)

### DICOM Data Preparation

A fake DICOM file will be prepared to test the performance.

In [5]:
temp_working_dir = tempfile.mkdtemp()

In [6]:
def create_multiframe_dicom(
    file_path,
    num_slices=128,
    pixel_array_shape=(1024, 1024),
    pixel_value_range=(0, 4095),
):
    # Create a new DICOM dataset
    file_meta = pydicom.dataset.FileMetaDataset()
    file_meta.MediaStorageSOPClassUID = pydicom.uid.generate_uid()
    file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    file_meta.ImplementationClassUID = pydicom.uid.PYDICOM_IMPLEMENTATION_UID
    file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    # Create the FileDataset instance (inherits from Dataset)
    ds = FileDataset(file_path, {}, file_meta=file_meta, preamble=b"\0" * 128)

    # Set some basic DICOM attributes
    ds.PatientName = "Test^Patient"
    ds.PatientID = "123456"
    ds.Modality = "CT"
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
    ds.StudyDate = datetime.date.today().strftime("%Y%m%d")
    ds.ContentDate = datetime.date.today().strftime("%Y%m%d")
    ds.StudyTime = datetime.datetime.now().strftime("%H%M%S")
    ds.ContentTime = datetime.datetime.now().strftime("%H%M%S")

    # Set the pixel data with random integers
    pixel_array = np.random.randint(
        pixel_value_range[0],
        pixel_value_range[1],
        (num_slices, *pixel_array_shape),
        dtype=np.uint16,
    )
    ds.Rows, ds.Columns = pixel_array_shape
    ds.NumberOfFrames = num_slices
    ds.PixelData = pixel_array.tobytes()
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15
    ds.PixelRepresentation = 0

    # Set multi-frame specific attributes
    ds.PerFrameFunctionalGroupsSequence = []
    for slice_index in range(num_slices):
        frame = Dataset()
        plane_position = Dataset()
        plane_position.ImagePositionPatient = [0, 0, slice_index]
        plane_orientation = Dataset()
        plane_orientation.ImageOrientationPatient = [1, 0, 0, 0, 1, 0]
        pixel_measures = Dataset()
        pixel_measures.SliceThickness = 1

        frame.PlanePositionSequence = [plane_position]
        frame.PlaneOrientationSequence = [plane_orientation]
        frame.PixelMeasuresSequence = [pixel_measures]
        ds.PerFrameFunctionalGroupsSequence.append(frame)

    ds.is_little_endian = True
    ds.is_implicit_VR = True

    ds.save_as(file_path)
    print(f"Multi-frame DICOM file created at: {file_path}")


# Example usage
example_dcm_path = os.path.join(temp_working_dir, "example.dcm")

create_multiframe_dicom(example_dcm_path)

Multi-frame DICOM file created at: /tmp/tmplw76c4_0/example.dcm


### Test DICOM Data Loading

In [7]:
def load_image_cpu(file_path):
    ds = pydicom.dcmread(file_path)
    pixel_array = ds.pixel_array
    return pixel_array

In [8]:
def load_image_gpu(file_path):
    # set defer_size to prevent reading the entire file
    dcm_read_data = pydicom.dcmread(file_path, defer_size="100 KB")

    # Extract relevant attributes
    rows = dcm_read_data.Rows
    columns = dcm_read_data.Columns
    bits_allocated = dcm_read_data.BitsAllocated
    samples_per_pixel = dcm_read_data.SamplesPerPixel
    number_of_frames = getattr(dcm_read_data, "NumberOfFrames", 1)
    pixel_representation = dcm_read_data.PixelRepresentation

    if bits_allocated == 8:
        dtype = cp.int8 if pixel_representation == 1 else cp.uint8
    elif bits_allocated == 16:
        dtype = cp.int16 if pixel_representation == 1 else cp.uint16
    elif bits_allocated == 32:
        dtype = cp.int32 if pixel_representation == 1 else cp.uint32
    else:
        raise ValueError("Unsupported BitsAllocated value")

    bytes_per_pixel = bits_allocated // 8
    total_pixels = rows * columns * samples_per_pixel * number_of_frames
    expected_pixel_data_length = total_pixels * bytes_per_pixel

    offset = dcm_read_data.get_item(0x7FE00010, keep_deferred=True).value_tell

    with kvikio.CuFile(file_path, "r") as f:
        buffer = cp.empty(expected_pixel_data_length, dtype=cp.int8)
        f.read(buffer, expected_pixel_data_length, offset)

    cupy_data_array = buffer.view(dtype).reshape((number_of_frames, rows, columns))

    return cupy_data_array

In [ ]:
# Measure Kvikio GPU loading time
# the saved outputs are run with a Tesla V100-PCIE-16GB GPU
start_gpu = timer()
img_gpu = load_image_gpu(example_dcm_path)
print(img_gpu.shape, img_gpu.mean())
end_gpu = timer()
gpu_time = end_gpu - start_gpu
print(f"Kvikio GPU loading time: {gpu_time:.4f} seconds")

(128, 1024, 1024) 2046.878812596202
Kvikio GPU loading time: 0.0975 seconds


In [10]:
# Measure CPU loading time
start_cpu = timer()
img_cpu = load_image_cpu(example_dcm_path)
print(img_cpu.shape, img_cpu.mean())
end_cpu = timer()
cpu_time = end_cpu - start_cpu
print(f"Normal CPU loading time: {cpu_time:.4f} seconds")

(128, 1024, 1024) 2046.878812596202
Normal CPU loading time: 0.3950 seconds


### validate cpu and gpu data are close

In [11]:
print(np.allclose(img_gpu, img_cpu))

True


### Cleanup tmp Directory

In [12]:
shutil.rmtree(temp_working_dir)